In [1]:
from multiprocessing import Pool, cpu_count
import pandas as pd
from main import process_url


In [2]:
cross = "cross-validation.csv"
df = pd.read_csv(cross)
domains = df.loc[:, 'site'].to_list()
urls = df.loc[:, 'url'].to_list()


In [3]:
with Pool(processes=cpu_count() - 1) as pool:
    results = pool.map(process_url, domains)


In [4]:
cMatrix = {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}
FN = []
FP = []
for result, url, domain in zip(results, urls, domains):
    if result is not None:
        if result == url:
            cMatrix["TP"] += 1
        else:
            cMatrix["FP"] += 1
            FP.append((result, url, domain))
    else:
        if pd.isna(url):
            cMatrix["TN"] += 1
        else:
            cMatrix["FN"] += 1
            FN.append((result, url, domain))


In [6]:
Precision = cMatrix['TP'] / (cMatrix['TP'] + cMatrix['FP'])
Recall = cMatrix['TP'] / (cMatrix['TP'] + cMatrix['FN'])
F1 = 2 * ((Precision * Recall) / (Precision + Recall))
Accuracy = (cMatrix['TP'] + cMatrix['TN']) / (
    cMatrix['TP'] + cMatrix['FP'] + cMatrix['TN'] + cMatrix['FN'])
Specificity = cMatrix['TN'] / (cMatrix['TN'] + cMatrix['FP'])
NPV = cMatrix['TN'] / (cMatrix['TN'] + cMatrix['FN'])
Balanced_accuracy = (Recall + Specificity) / 2
print("Confusion matrix:", cMatrix, "\n")
print(f"{Precision=:.2f}")
print(f"{Recall=:.2f}")
print(f"{F1=:.2f}")
print(f"{Accuracy=:.2f}")
if cMatrix['TN'] != 0:
    print(f"{Specificity=:.2f}")
    print(f"{NPV=:.2f}")
    print(f"{Balanced_accuracy=:.2f}")


Confusion matrix: {'TP': 52, 'FP': 19, 'TN': 6, 'FN': 3} 

Precision=0.73
Recall=0.95
F1=0.83
Accuracy=0.72
Specificity=0.24
NPV=0.67
Balanced_accuracy=0.59


In [5]:
for result, url, domain in FN:
    print("Got:", result)
    print("Cor:", url)
    print(domain, "FN")
    print()
for result, url, domain in FP:
    print("Got:", result)
    print("Cor:", url)
    print(domain, "FP")
    print()


Got: None
Cor: https://siteground.com/img/apple-touch-icon.png
siteground.com FN

Got: None
Cor: https://cdn.cookielaw.org/logos/17e5cb00-ad90-47f5-a58d-77597d9d2c16/d44e374b-e570-4884-9441-33c0ccae5431/959d0f3c-d044-46db-bc43-cbca0284a92d/NBCU_logo.png
ivillage.com FN

Got: None
Cor: https://onstar.com/content/dam/onstar/na/us/en/navigation/01-images/os-header-logo-shared.svg
onstar.com FN

Got: https://images.ctfassets.net/00i767ygo3tc/3NIN3TyNgzYx5YYlb5Hl8z/110981e92c9d86eae0a773356e6333e9/small-business-owner-using-a-free-video-maker-to-capture-attention-and-engage-audiences.webp?w=460&h=430&q=80
Cor: https://animoto.com/icons/icon-512x512.png?v=f7bad0df2a4af8688773dca5ee0b1ed6
animoto.com FP

Got: data:image/svg+xml,%3Csvg%20xmlns=%22http://www.w3.org/2000/svg%22%20viewBox=%220%200%20210%20140%22%3E%3C/svg%3E
Cor: https://enterprise.efax.com/wp-content/themes/enterprise-efax-child/_ui/img/logo-efax-corp-white.png
efaxcorporate.com FP

Got: https://vanilla.futurecdn.net/cyclingnews